# 精调万分--脊柱分割


In [7]:
# !git clone https://github.com/jumbojing/MedSAM.git

# !git clone https://github.com/hitachinsk/SAMed.git

From https://github.com/jumbojing/MedSAM
 * branch            HEAD       -> FETCH_HEAD
Already up to date.


In [1]:
%cd /kaggle/working/MedSAM
!git pull https://github.com/jumbojing/MedSAM.git
!pip install -e.

/kaggle/working/MedSAM
From https://github.com/jumbojing/MedSAM
 * branch            HEAD       -> FETCH_HEAD
Already up to date.
Obtaining file:///kaggle/working/MedSAM
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.6 MB/s eta 0:00:00a 0:00:01
  Running setup.py develop for medsam


In [2]:
%cd /kaggle/working/SAMed
!git pull https://github.com/jumbojing/SAMed.git
# !pip install -e.

/kaggle/working/SAMed
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 692 bytes | 692.00 KiB/s, done.
From https://github.com/jumbojing/SAMed
 * branch            HEAD       -> FETCH_HEAD
Updating e6376b5..513417e
Fast-forward
 README.md | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [2]:
    

# !pip install monai

# %% set up environment
import numpy as np
import matplotlib.pyplot as plt
import os
join = os.path.join
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
import monai
from segment_anything import SamPredictor, sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
# from segment_anything.imgLoraEncoder import LoRA_Sam

# !pip install SimpleITK

import SimpleITK as sitk
from collections import defaultdict as dfDic
# def fdic():
#     return dfDic(fdic)
from matplotlib.colors import ListedColormap, Normalize
#%% import packages
import os
join = os.path.join
from skimage import transform # , io, segmentation
from tqdm import tqdm
import argparse
import sys
sys.argv = ['run.py']

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [25]:

def samShow(img=None, masks=None, bxCapt=['',''], points=None,
            boxes=None, fSize=(10,10), dpi = 66, pad=20, opacity=0.4):
    def __showMsk(masks=masks, ax=None, opacity=opacity):
        labels = np.unique(masks)[1:]
        print(labels)
        # if labels > 0: # or not np.isnan(labels):
        for lb in labels:
            level = LBLVS[lb]
            vClr = COLORS[int(lb)]
            mask = np.array(masks==lb)
            h, w = mask.shape[-2:]
            vClr = np.array(vClr.tolist() + [opacity])
            mask_image = mask.reshape(h, w, 1) * vClr.reshape(1, 1, -1)
            ax.imshow(mask_image)

            # ax.patches.clear()
            yInd, xInd = np.where(mask==1)
            # print(len(yInd),len(xInd))
            x0, x1 = np.min(xInd), np.max(xInd)
            y0, y1 = np.min(yInd), np.max(yInd)
            ax.add_patch(plt.Rectangle((x0, y0), x1-x0, y1-y0, edgecolor=vClr, facecolor=(0,0,0,0), lw=3))
            if pad is not None:
                h, w = mask.shape
                xP0 = max(0, x0 - np.random.randint(0, pad))
                xP1 = min(w, x1 + np.random.randint(0, pad))
                yP0 = max(0, y0 - np.random.randint(0, pad))
                yP1 = min(h, y1 + np.random.randint(0, pad))
                ax.add_patch(plt.Rectangle((xP0,yP0),xP1-xP0,yP1-yP0,edgecolor=vClr,facecolor=(0,0,0,0), lw=2, linestyle='dashdot'))
            locLb = level + ' ' + bxCapt[0]
            iouLb = f' {bxCapt[1]}'
            plt.text(x0, y0, locLb+iouLb, fontsize=12, color='white',backgroundcolor=vClr)
            plt.text(x0, y0, locLb+iouLb, fontsize=12, color='white',backgroundcolor=vClr)
            # plt.text(x1, y1, 'A\n R', fontsize=12, color='white',backgroundcolor=vClr)

    # def __showPs(coords, labels, ax, marker_size=375):
    #     pos_points = coords[labels==1]
    #     neg_points = coords[labels==0]
    #     ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    #     ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

    # def __showBox(box, ax):
    #     x0, y0 = box[0], box[1]
    #     w, h = box[2] - box[0], box[3] - box[1]
    #     ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))
    if fSize is None:
        if img is not None:
            h, w = img.shape[1:3]
        elif masks is not None:
            h, w = masks.shape[1:3]
        fSize = (w/dpi, h/dpi)
    else:
        h,w = np.asarray(fSize)*dpi

    def callback(zI_to_S=None):
        plt.figure(figsize=fSize, dpi=dpi)
        # plt.gca().clear()
        if zI_to_S is None:
            if img is not None:
                plt.imshow(img, cmap='gray')
            if masks is not None:
                # plt.gca().patches.clear()
                __showMsk(masks, plt.gca())
            # plt.imshow(img, cmap='gray')
        else:
            if img is not None:
                plt.imshow(img[zI_to_S,...], cmap='gray')
            if masks is not None:
                __showMsk(masks[zI_to_S,...], plt.gca()) # gca: get current axis
        # if points is not None:
        #     __showPs(points, labels, plt.gca())
        # if boxes is not None:
        #     for box in [boxes,[boxes]][int(len(boxes)==1)]:
        #         __showBox(box, plt.gca())
        plt.text(0,0,'O L>>>R\nP\nV\nV\nA', ha='left',va='top',fontsize=16, color='white')
        plt.axis('off')
        plt.show()

        return plt

    imgs = [masks,img][img is not None]
    print(imgs.shape)
    if imgs.ndim == 2:
        callback()
    elif imgs.ndim in[3,4]:
        if imgs.shape[2] in [1,3]:
            # imgs添加一个维度到第一个位置且值为2
            callback()
        else:
            zAxis = imgs.shape[0]-1
            # print(imgs.shape)
            interact(callback, zI_to_S=(0, zAxis))



In [3]:
%cd /kaggle/working/SAMed
from segment_anything import SamPredictor, sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide

/kaggle/working/SAMed


In [9]:
%cd ..
import os
os.mkdir('medSamSpine')
os.mkdir('medSamSpine/images')
os.mkdir('medSamSpine/lables')
os.mkdir('medSamSpine/data')
os.mkdir('medSamSpine/data/Npz_files')
# os.mkdir('medSamSpine/results_folder')

/kaggle/working


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508 in run_code        │
│                                                                                                  │
│   3505 │   │   │   │   if async_:                                                                │
│   3506 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3507 │   │   │   │   else:                                                                     │
│ ❱ 3508 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3509 │   │   │   finally:                                                                      │
│   3510 │   │   │   │   # Reset our crash handler in place                                        │
│   3511 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 get_ipython().run_line_magic('cd', '..')                                                     │
│   2 import os                                                                                    │
│ ❱ 3 os.mkdir('medSamSpine')                                                                      │
│   4 os.mkdir('medSamSpine/images')                                                               │
│   5 os.mkdir('medSamSpine/lables')                                                               │
│   6 os.mkdir('medSamSpine/data')                                                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileExistsError: [Errno 17] File exists: 'medSamSpine'

In [6]:
model_type = 'vit_b'
checkpoint = '/kaggle/input/segment-anything/pytorch/vit-b/1/model.pth'
device = 'cuda'
# 精调炉
# from segment_anything import SamPredictor, sam_model_registry
sam = sam_model_registry[model_type](checkpoint=checkpoint)
sam.to(device)
sam.train();

In [7]:
imgf = f'/kaggle/input/vescos-dataset/verse564/verse564_CT-iso.nii'
mskf = f'/kaggle/input/vescos-dataset/verse564/verse564_CT-iso_seg.nii'
# imgSk = sitk.ReadImage(imgf)
# mskSk = sitk.ReadImage(mskf)

In [8]:
# def preprocess_ct(gt_path, nii_path, gt_name, image_name, label_id, image_size, sam_model, device):
gt_sitk = sitk.ReadImage(mskf)
gt_data = sitk.GetArrayFromImage(gt_sitk)
img_sitk = sitk.ReadImage(imgf)
# image_data = sitk.GetArrayFromImage(img_sitk)

In [13]:
imgSk=readImgSk(imgf)

NameError: name 'readImgSk' is not defined

In [10]:
class NpzDataset(Dataset): 
    def __init__(self, data_root, lv=None):
        if isinstance(data_root, dict):
            self.lv=lv
            self.ori_gts = data_root[self.lv]['gts']
            self.imgEmds = data_root[self.lv]['imgEmds']
        elif os.path.isdir(data_root):
            self.data_root = data_root
            self.npz_files = sorted(os.listdir(self.data_root)) 
            self.npz_data = [np.load(join(data_root, f)) for f in self.npz_files]
        # this implementation is ugly but it works (and is also fast for feeding data to GPU) if your server has enough RAM
        # as an alternative, you can also use a list of npy files and load them one by one
            self.ori_gts = np.vstack([d['gts'] for d in self.npz_data])
            self.imgEmds = np.vstack([d['imgEmds'] for d in self.npz_data])
        elif data_root.endswith(f'\.npz'):
            self.data_root = np.load(data_root)
#             self.lv = self.data_root[4:-4]
            self.ori_gts = self.data_root['gts']
            self.imgEmds = self.data_root['imgEmds']
        
        print(f"{self.imgEmds.shape=}, {self.ori_gts.shape=}")
    
    def __len__(self):
        return self.ori_gts.shape[0]

    def __getitem__(self, index):
        img_embed = self.imgEmds[index]
        gt2D = self.ori_gts[index]
        y_indices, x_indices = np.where(gt2D > 0)
        x_min, x_max = np.min(x_indices), np.max(x_indices)
        y_min, y_max = np.min(y_indices), np.max(y_indices)
        # add perturbation to bounding box coordinates
        H, W = gt2D.shape
        x_min = max(0, x_min - np.random.randint(0, 20))
        x_max = min(W, x_max + np.random.randint(0, 20))
        y_min = max(0, y_min - np.random.randint(0, 20))
        y_max = min(H, y_max + np.random.randint(0, 20))
        bboxes = np.array([x_min, y_min, x_max, y_max])
        # convert img embedding, mask, bounding box to torch tensor
        return torch.tensor(img_embed).float(), torch.tensor(gt2D[None, :,:]).long(), torch.tensor(bboxes).float()


In [11]:
from sam_lora_image_encoder import LoRA_Sam

In [12]:
lora=LoRA_Sam(sam,4)

In [13]:
loraEncoder=lora.sam.image_encoder

In [29]:
result = lora.sam.image_encoder(torch.rand(size=(1,3,1024,1024)))
print(result.shape)

torch.Size([1, 256, 64, 64])


In [45]:
def preCT(imgSk, gt_data, npz='',sam_model=sam, lbMin=14., lvs=None, imgEncoder=loraEncoder):
    image_data = sitk.GetArrayFromImage(imgSk)
    gtDic = {}
    if lvs is None:
        labels = np.unique(gt_data > 0)[1:]  
    else:
        labels=[]
        for val in lvs:
            labels+=([k for k, v in LBLVS.items() if v == val])
    for lb in labels:
        if lb > lbMin:  
            mask = np.uint8(gt_data==lb)[1:]
        if np.sum(mask) > 0:  
            gtDic[lb] = mask
    dic = dfDic(dict)
    # gt_data = np.uint8(gt_data!=0)
    count=1
    for id, gt_data in gtDic.items():
        if gt_data.size > 0:
            assert np.max(gt_data)==1 and np.unique(gt_data).shape[0]==2, 'ground truth should be binary'
#             assert np.sum(gt_data)==0, 'ground truth is empty'
            # nii preprocess start
            lower_bound = 0
            upper_bound = 250
            image_data_pre = np.clip(image_data, lower_bound, upper_bound) # 目的是将图像的像素值限制在一定范围内
            image_data_pre = (image_data_pre - np.min(image_data_pre))/(np.max(image_data_pre)-np.min(image_data_pre))*255.0 # 归一化
            image_data_pre[image_data==0] = 0 # 将图像中的0值像素置为0
            image_data_pre = np.uint8(image_data_pre) # 转为uint8

            z_index, _, _ = np.where(gt_data>0) # z轴上的索引
            z_min, z_max = np.min(z_index), np.max(z_index)
            imgs=[]
            gts=[]
            imgEmds=[]
            ipImgs=[]
            for i in range(z_min, z_max):
                gt_slice_i = gt_data[i,:,:]
                if np.sum(gt_slice_i)>0:                    
                    gt_slice_i = transform.resize(gt_slice_i, (256, 256), order=0, preserve_range=True, mode='constant', anti_aliasing=True) # 重采样: 保持原图像的像素值, 重采样到256x256, 最近邻插值, 常数填充, 抗锯齿
#                     if np.sum(gt_slice_i)>1:
                        # resize img_slice_i to 256x256
                    img_slice_i = transform.resize(image_data_pre[i,:,:], (256,256), order=3, preserve_range=True, mode='constant', anti_aliasing=True)
                    # convert to three channels
                    img_slice_i = np.uint8(np.repeat(img_slice_i[:,:,None], 3, axis=-1))
                    assert len(img_slice_i.shape)==3 and img_slice_i.shape[2]==3, 'image should be 3 channels'
                    assert img_slice_i.shape[0]==gt_slice_i.shape[0] and img_slice_i.shape[1]==gt_slice_i.shape[1], 'image and ground truth should have the same size'
                    imgs.append(img_slice_i)
#                         assert np.sum(gt_slice_i)>0, f'the gt_{i} is empty' # 保证ground truth中有超过100个像素
                    gts.append(gt_slice_i)
                    if sam_model is not None:
                        sam_transform = ResizeLongestSide(sam_model.imgEncoder.img_size)
                        resize_img = sam_transform.apply_image(img_slice_i)
                                                # resized_shapes.append(resize_img.shape[:2])
                        resize_img_tensor = torch.as_tensor(resize_img.transpose(2, 0, 1)).to(device)
                                                # model input: (1, 3, 1024, 1024)
                        input_image = sam_model.preprocess(resize_img_tensor[None,:,:,:]) # (1, 3, 1024, 1024)
                        print(np.array(input_image).shape)
                        ipImgs.append(input_image)
                        assert input_image.shape == (1, 3, sam_model.imgEncoder.img_size, 
                                                     sam_model.imgEncoder.img_size), 'input image should be resized to 1024*1024'
                                                # input_imgs.append(input_image.cpu().numpy()[0])
                        embedding = sam_model.imgEncoder(input_image)
                        print(embedding.shape)
                        imgEmds.append(embedding)
                            
            print(np.asarray(imgEmds).shape)            
#             break
            if len(imgs)>1:
                    lv = LBLVS[id]        
                    imgs = np.stack(imgs, axis=0) # (n, 256, 256, 3)
                    ipImgs = np.stack(np.array(ipImgs), axis=0)
                    if len(imgEmds)>0:
                        imgEmds = np.stack(imgEmds, axis=0) # (n, 1, 256, 64, 64)
                    gts = np.stack(gts, axis=0) # (n, 256, 256)
                    dic[lv] = dict(imgs=np.asarray(imgs), gts=np.asarray(gts), imgEmds=np.asarray(imgEmds), ipImgs=np.asarray(ipImgs))
                    if npz!='':
                        np.savez_compressed(f'{npz}{lv}.npz', dic[lv])
            count+=1
            if count>3:
                break
    return dic


In [46]:
dicT11=preCT(img_sitk, gt_data,sam_model=None,lvs=['T11'])

(0,)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508 in run_code        │
│                                                                                                  │
│   3505 │   │   │   │   if async_:                                                                │
│   3506 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3507 │   │   │   │   else:                                                                     │
│ ❱ 3508 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3509 │   │   │   finally:                                                                      │
│   3510 │   │   │   │   # Reset our crash handler in place                                        │
│   3511 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 dicT11=preCT(img_sitk, gt_data,sam_model=None,lvs=['T11'])                                   │
│   2                                                                                              │
│                                                                                                  │
│ in preCT:71                                                                                      │
│                                                                                                  │
│   68 │   │   │   if len(imgs)>1:                                                                 │
│   69 │   │   │   │   │   lv = LBLVS[id]                                                          │
│   70 │   │   │   │   │   imgs = np.stack(imgs, axis=0) # (n, 256, 256, 3)                        │
│ ❱ 71 │   │   │   │   │   ipImgs = np.stack(np.array(ipImgs), axis=0)                             │
│   72 │   │   │   │   │   if len(imgEmds)>0:                                                      │
│   73 │   │   │   │   │   │   imgEmds = np.stack(imgEmds, axis=0) # (n, 1, 256, 64, 64)           │
│   74 │   │   │   │   │   gts = np.stack(gts, axis=0) # (n, 256, 256)                             │
│ in stack:180                                                                                     │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/numpy/core/shape_base.py:422 in stack                    │
│                                                                                                  │
│   419 │                                                                                          │
│   420 │   arrays = [asanyarray(arr) for arr in arrays]                                           │
│   421 │   if not arrays:                                                                         │
│ ❱ 422 │   │   raise ValueError('need at least one array to stack')                               │
│   423 │                                                                                          │
│   424 │   shapes = {arr.shape for arr in arrays}                                                 │
│   425 │   if len(shapes) != 1:                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: need at least one array to stack

In [27]:
np.asarray(dicT11['T11']['imgEmds']).shape

(0,)

In [29]:
lImgs = loraEncoder(dicT11['T11']['imgs'][3])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508 in run_code        │
│                                                                                                  │
│   3505 │   │   │   │   if async_:                                                                │
│   3506 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3507 │   │   │   │   else:                                                                     │
│ ❱ 3508 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3509 │   │   │   finally:                                                                      │
│   3510 │   │   │   │   # Reset our crash handler in place                                        │
│   3511 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 lImgs = loraEncoder(dicT11['T11']['imgs'][3])                                                │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /kaggle/working/MedSAM/segment_anything/modeling/image_encoder.py:107 in forward                 │
│                                                                                                  │
│   104 │   │   )                                                                                  │
│   105 │                                                                                          │
│   106 │   def forward(self, x: torch.Tensor) -> torch.Tensor:                                    │
│ ❱ 107 │   │   x = self.patch_embed(x)                                                            │
│   108 │   │   if self.pos_embed is not None:                                                     │
│   109 │   │   │   x = x + self.pos_embed                                                         │
│   110                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _gl

In [26]:
samShow(dicT11['T11']['imgs'],dicT11['T11']['gts'])

(51, 256, 256, 3)


interactive(children=(IntSlider(value=25, description='zI_to_S', max=50), Output()), _dom_classes=('widget-int…

In [ ]:
dicT12=preCT(img_sitk, gt_data,sam_model=sam_model,lvs=['T12'])

In [29]:
#%% create a dataset class to load npz data and return back image embeddings and ground truth
class NpzDataset(Dataset): 
    def __init__(self, data_root, lv=None):
        if isinstance(data_root, dict):
            self.lv=lv
            self.ori_gts = data_root[self.lv]['gts']
            self.imgEmds = data_root[self.lv]['imgEmds']
        elif os.path.isdir(data_root):
            self.data_root = data_root
            self.npz_files = sorted(os.listdir(self.data_root)) 
            self.npz_data = [np.load(join(data_root, f)) for f in self.npz_files]
        # this implementation is ugly but it works (and is also fast for feeding data to GPU) if your server has enough RAM
        # as an alternative, you can also use a list of npy files and load them one by one
            self.ori_gts = np.vstack([d['gts'] for d in self.npz_data])
            self.imgEmds = np.vstack([d['imgEmds'] for d in self.npz_data])
        elif data_root.endswith(f'\.npz'):
            self.data_root = np.load(data_root)
#             self.lv = self.data_root[4:-4]
            self.ori_gts = self.data_root['gts']
            self.imgEmds = self.data_root['imgEmds']
        
        print(f"{self.imgEmds.shape=}, {self.ori_gts.shape=}")
    
    def __len__(self):
        return self.ori_gts.shape[0]

    def __getitem__(self, index):
        img_embed = self.imgEmds[index]
        gt2D = self.ori_gts[index]
        y_indices, x_indices = np.where(gt2D > 0)
        x_min, x_max = np.min(x_indices), np.max(x_indices)
        y_min, y_max = np.min(y_indices), np.max(y_indices)
        # add perturbation to bounding box coordinates
        H, W = gt2D.shape
        x_min = max(0, x_min - np.random.randint(0, 20))
        x_max = min(W, x_max + np.random.randint(0, 20))
        y_min = max(0, y_min - np.random.randint(0, 20))
        y_max = min(H, y_max + np.random.randint(0, 20))
        bboxes = np.array([x_min, y_min, x_max, y_max])
        # convert img embedding, mask, bounding box to torch tensor
        return torch.tensor(img_embed).float(), torch.tensor(gt2D[None, :,:]).long(), torch.tensor(bboxes).float()


In [33]:
# %% test dataset class and dataloader
# npz_tr_path = r'/kaggle/input/npzdata564'
demo_dataset = NpzDataset(dicT11, 'T11')
demo_dataloader = DataLoader(demo_dataset, batch_size=8, shuffle=True)
for imgEmd,gt2D, bboxes in demo_dataloader:
    # img_embed: (B, 256, 64, 64), gt2D: (B, 1, 256, 256), bboxes: (B, 4)
    print(f"{imgEmd.shape=}, {gt2D.shape=}, {bboxes.shape=}")
    break

self.imgEmds.shape=(0,), self.ori_gts.shape=(51, 256, 256)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508 in run_code        │
│                                                                                                  │
│   3505 │   │   │   │   if async_:                                                                │
│   3506 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3507 │   │   │   │   else:                                                                     │
│ ❱ 3508 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3509 │   │   │   finally:                                                                      │
│   3510 │   │   │   │   # Reset our crash handler in place                                        │
│   3511 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2 # npz_tr_path = r'/kaggle/input/npzdata564'                                                  │
│   3 demo_dataset = NpzDataset(dicT11, 'T11')                                                     │
│   4 demo_dataloader = DataLoader(demo_dataset, batch_size=8, shuffle=True)                       │
│ ❱ 5 for imgEmd,gt2D, bboxes in demo_dataloader:                                                  │
│   6 │   # img_embed: (B, 256, 64, 64), gt2D: (B, 1, 256, 256), bboxes: (B, 4)                    │
│   7 │   print(f"{imgEmd.shape=}, {gt2D.shape=}, {bboxes.shape=}")                                │
│   8 │   break                                                                                    │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:634 in __next__           │
│                                                                                                  │
│    631 │   │   │   if self._sampler_iter is None:                                                │
│    632 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    633 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  634 │   │   │   data = self._next_data()                                                      │
│    635 │   │   │   self._num_yielded += 1                                                        │
│    636 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    637 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:678 in _next_data         │
│                                                                                                  │
│    675 │                                                                                         │
│    676 │   def _next_data(self):                                                                 │
│    677 │   │   index = self._next_index()  # may raise StopIteration                             │
│ ❱  678 │   │   data = self._dataset_fetcher.fetch(index)  # may raise StopIteration              │
│    679 │   │   if self._pin_memory:                                                              │
│    680 │   │   │   data = _utils.pin_memory.pin_memory(data, self._pin_memory_device)            │
│    681 │   │   return data                                 

In [ ]:
ls

In [20]:
# %% set up model for fine-tuning 
# train data path
# npz_tr_path = r'/kaggle/input/npzdata564/'
work_dir = '/kaggle/working'
task_name = 'CT_spineVerSe'
# prepare SAM model
model_type = 'vit_b'
checkpoint = '/kaggle/input/segment-anything/pytorch/vit-b/1/model.pth'
device = 'cuda:0'
model_save_path = 'medSamSpine'
os.makedirs(model_save_path, exist_ok=True)
sam_model = sam_model_registry[model_type](checkpoint=checkpoint).to(device)
sam_model.train()

# Set up the optimizer, hyperparameter tuning will improve performance here
optimizer = torch.optim.Adam(sam_model.mask_decoder.parameters(), lr=1e-5, weight_decay=0)
seg_loss = monai.losses.DiceCELoss(sigmoid=True, squared_pred=True, reduction='mean')


In [21]:
#%% train
num_epochs = 2
losses = []
best_loss = 1e10
train_dataset = NpzDataset(dicT11, 'T11')
dataTn = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [ ]:
len(dataTn[6:13]

In [22]:
# for step, (image_embed, gt2D, bboxes) in enumerate(tqdm(train_dataloader)):
for step, (img, gt2D, bboxes) in enumerate(dataTn):
    print(f"{img.shape=}, {gt2D.shape=}, {bboxes.shape=}")
    break
    

img.shape=torch.Size([16, 256, 256, 3]), gt2D.shape=torch.Size([16, 1, 256, 256]), bboxes.shape=torch.Size([16, 4])


In [ ]:
np.arange(6,10)

In [ ]:
# Load latest or best checkpoint weights
sam_model.load_state_dict(torch.load(join(model_save_path, 'sam_model_latest.pth')))

# Set model to train mode 
sam_model.train()

num_epochs=50
# for epoch in range(num_epochs):
#     ...
    
#     # Train loop
#     for step, batch in enumerate(train_loader):
#         ...
        
#     # Validation loop
#     for step, batch in enumerate(val_loader):
#         ...
        
#     # Save latest and best models
#     ...


In [23]:
msPrd={}
for epoch in range(num_epochs):
    epoch_loss = 0
    # train
    for step, (image_embed, gt2D, bboxes) in enumerate(tqdm(dataTn)):
        # 嵌图, 不算梯度
        with torch.no_grad():
            box_np = bboxes.numpy()
            sam_trans = ResizeLongestSide(sam_model.image_encoder.img_size)
            box = sam_trans.apply_boxes(box_np, (gt2D.shape[-2], gt2D.shape[-1]))
            box_torch = torch.as_tensor(box, dtype=torch.float, device=device)
            if len(box_torch.shape) == 2:
                box_torch = box_torch[:, None, :] # (B, 1, 4)
            # get prompt embeddings 
            sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
                points=None,
                boxes=box_torch,
                masks=None,
            )
#         print(f"{image_embed.shape=}")
#         print(f"{sam_model.prompt_encoder.get_dense_pe().shape=}")
#         print(f"{sparse_embeddings.shape=}")
#         print(f"{dense_embeddings.shape=}")
        mask_predictions, _ = sam_model.mask_decoder(
            image_embeddings=image_embed.to(device), # (B, 256, 64, 64)          
            image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
            sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
            dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
            multimask_output=False,
          )
        msPrd[step]=mask_predictions
        loss = seg_loss(mask_predictions, gt2D.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    epoch_loss /= step
    losses.append(epoch_loss)
    print(f'EPOCH: {epoch}, Loss: {epoch_loss}')
    # save the latest model checkpoint
    torch.save(sam_model.state_dict(), join(model_save_path, 'sam_model_latest.pth'))
    # save the best model
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(sam_model.state_dict(), join(model_save_path, 'sam_model_best.pth'))

  0%|          | 0/4 [00:01<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3508 in run_code        │
│                                                                                                  │
│   3505 │   │   │   │   if async_:                                                                │
│   3506 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3507 │   │   │   │   else:                                                                     │
│ ❱ 3508 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3509 │   │   │   finally:                                                                      │
│   3510 │   │   │   │   # Reset our crash handler in place                                        │
│   3511 │   │   │   │   sys.excepthook = old_excepthook                                           │
│                                                                                                  │
│ in <module>:24                                                                                   │
│                                                                                                  │
│   21 #         print(f"{sam_model.prompt_encoder.get_dense_pe().shape=}")                        │
│   22 #         print(f"{sparse_embeddings.shape=}")                                              │
│   23 #         print(f"{dense_embeddings.shape=}")                                               │
│ ❱ 24 │   │   mask_predictions, _ = sam_model.mask_decoder(                                       │
│   25 │   │   │   image_embeddings=image_embed.to(device), # (B, 256, 64, 64)                     │
│   26 │   │   │   image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)            │
│   27 │   │   │   sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)                       │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /kaggle/working/MedSAM/segment_anything/modeling/mask_decoder.py:94 in forward                   │
│                                                                                                  │
│    91 │   │     torch.Tensor: batched predicted masks                                            │
│    92 │   │     torch.Tensor: batched predictions of mask quality                                │
│    93 │   │   """                                                                                │
│ ❱  94 │   │   masks, iou_pred = self.predict_masks(                                              │
│    95 │   │   │   image_embeddings=image_embeddings,                                             │
│    96 │   │   │   image_pe=image_pe,                                                             │
│    97 │   │   │   sparse_prompt_embeddings=sparse_prompt_em

In [ ]:
 if image_embeddings.shape[0] != tokens.shape[0]: 
     src = torch.repeat_interleave(image_embeddings, tokens.shape[0], dim=0) 
 else: 
     src = image_embeddings 
 src = src + dense_prompt_embeddings 
 pos_src = torch.repeat_interleave(image_pe, tokens.shape[0], dim=0) 
 b, c, h, w = src.shape 

In [ ]:
# plot loss
plt.plot(losses)
plt.title('Dice + Cross Entropy Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show() # comment this line if you are running on a server
# plt.savefig(join(model_save_path, 'train_loss.png'))
plt.close()

In [ ]:
#%% compare the segmentation results between the original SAM model and the fine-tuned model
# load the original SAM model
# ori_sam_model = sam_model_registry[model_type](checkpoint=checkpoint).to(device)
# ori_sam_predictor = SamPredictor(ori_sam_model)
# npz_ts_path = 'data/Npz_files/CT_Abd-Gallbladder/test'
# test_npzs = sorted(os.listdir(npz_ts_path))
# random select a test case
imgs = dicT12['T12']['imgs']
gts = dicT12['T12']['gts']

In [ ]:
samShow(imgs, gts)

In [ ]:
# load the original SAM model
ori_sam_model = sam_model_registry[model_type](checkpoint=checkpoint).to(device)
ori_sam_predictor = SamPredictor(ori_sam_model)
# npz = np.load(join(npz_ts_path, test_npzs[npz_idx]))
# Generate random permutation of 0 to 51
# idxs = np.random.permutation(52)  

# Shuffle imgs by indexing with idxs

# (52, 256, 256, 3)

imgs = dicT12['T12']['imgs']
gts = dicT12['T12']['gts']
idxs = np.random.randint(0, len(gts))
imgSs = imgs[idxs]  
gtSs = gts[idxs]  
print(f'{imgs.shape=}\n{gts.shape=}')

In [ ]:
def get_bbox_from_mask(mask):
    '''Returns a bounding box from a mask'''
    y_indices, x_indices = np.where(mask > 0)
    x_min, x_max = np.min(x_indices), np.max(x_indices)
    y_min, y_max = np.min(y_indices), np.max(y_indices)
    gtBbx = np.array([x_min, y_min, x_max, y_max])
    # add perturbation to bounding box coordinates
    H, W = mask.shape
    xP_min = max(0, x_min - np.random.randint(0, 20))
    xP_max = min(W, x_max + np.random.randint(0, 20))
    yP_min = max(0, y_min - np.random.randint(0, 20))
    yP_max = min(H, y_max + np.random.randint(0, 20))
    pdBbx=np.array([xP_min, yP_min, xP_max, yP_max])
    return gtBbx,pdBbx

ori_sam_segs = []
medsam_segs = []
gtBbxes = []
pdBbxes = []
for img, gt in zip(imgs, gts):
    gtBbx,bbox = get_bbox_from_mask(gt)
    gtBbxes.append(gtBbx)
    pdBbxes.append(bbox)
    # predict the segmentation mask using the original SAM model
    ori_sam_predictor.set_image(img)
    ori_sam_seg, _, _ = ori_sam_predictor.predict(point_coords=None, box=bbox, multimask_output=False)
    ori_sam_segs.append(ori_sam_seg[0])
    
    # predict the segmentation mask using the fine-tuned model
    H, W = img.shape[:2]
    resize_img = sam_trans.apply_image(img)
    resize_img_tensor = torch.as_tensor(resize_img.transpose(2, 0, 1)).to(device)
    input_image = sam_model.preprocess(resize_img_tensor[None,:,:,:]) # (1, 3, 1024, 1024)
    with torch.no_grad():
        image_embedding = sam_model.image_encoder(input_image.to(device)) # (1, 256, 64, 64)
        # convert box to 1024x1024 grid
        bbox = sam_trans.apply_boxes(bbox, (H, W))
        box_torch = torch.as_tensor(bbox, dtype=torch.float, device=device)
        if len(box_torch.shape) == 2:
            box_torch = box_torch[:, None, :] # (B, 1, 4)
        
        sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
            points=None,
            boxes=box_torch,
            masks=None,
        )
        medsam_seg_prob, _ = sam_model.mask_decoder(
            image_embeddings=image_embedding.to(device), # (B, 256, 64, 64)
            image_pe=sam_model.prompt_encoder.get_dense_pe(), # (1, 256, 64, 64)
            sparse_prompt_embeddings=sparse_embeddings, # (B, 2, 256)
            dense_prompt_embeddings=dense_embeddings, # (B, 256, 64, 64)
            multimask_output=False,
            )
        medsam_seg_prob = torch.sigmoid(medsam_seg_prob)
        # convert soft mask to hard mask
        medsam_seg_prob = medsam_seg_prob.cpu().numpy().squeeze()
        medsam_seg = (medsam_seg_prob > 0.5).astype(np.uint8)
        medsam_segs.append(medsam_seg)

In [ ]:
results = []
for batch_size in batch_sizes:
    with torch.autograd.profiler.profile() as prof:
        # Forward and backward pass with batch_size
    runtime = prof.total_average()
    memory = prof.total_allocated_bytes.item() / (1024 ** 2)
    results.append((batch_size, runtime, memory))

In [ ]:
# 测试批量大小
batch_sizes = [8, 16, 32, 64, 128, 256, 512, 1024]  # 批量大小
try:
    results = profile(model.parameters(), model, n=3, device=device) 
except Exception as e:  
    LOGGER.warning(f'{prefix}{e}')  

# 拟合解决方案
y = [x[2] for x in results if x]  # 内存 [2]
p = np.polyfit(batch_sizes[:len(y)], y, deg=1)  # 一次多项式拟合
b = int((f * fraction - p[1]) / p[0])  # y 拦截 (最佳批量大小)
if None in results:  
    i = results.index(None)  # 第一个失败索引
    if b >= batch_sizes[i]:  
        b = batch_sizes[max(i - 1, 0)]  

fraction = (np.polyval(p, b) + r + a) / t  # 实际预测的分数
LOGGER.info(f'{prefix}使用批量大小 {b} 对于 {d} {t * fraction:.2f}G/{t:.2f}G ({fraction * 100:.0f}%) ✅')
return b

In [ ]:
cd ..

In [ ]:
%cd MedSAM
from utils.SurfaceDice import compute_dice_coefficient

In [ ]:
#%% compute the DSC score
from utils.SurfaceDice import compute_dice_coefficient
ori_sam_segs = np.stack(ori_sam_segs, axis=0)
medsam_segs = np.stack(medsam_segs, axis=0)
ori_sam_dsc = compute_dice_coefficient(gts>0, ori_sam_segs>0)
medsam_dsc = compute_dice_coefficient(gts>0, medsam_segs>0)
print('Original SAM DSC: {:.4f}'.format(ori_sam_dsc), 'MedSAM DSC: {:.4f}'.format(medsam_dsc))


In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([251/255, 252/255, 30/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='blue', facecolor=(0,0,0,0), lw=2))    


img_id = int(imgs.shape[0]//2)  # np.random.randint(imgs.shape[0])
_, axs = plt.subplots(1, 3, figsize=(25, 25))
axs[0].imshow(imgs[img_id])
show_mask(gts[img_id], axs[0])
# show_box(box_np[img_id], axs[0])
# axs[0].set_title('Mask with Tuned Model', fontsize=20)
axs[0].axis('off')

axs[1].imshow(imgs[img_id])
show_mask(ori_sam_segs[img_id], axs[1])
show_box(gtBbxes[img_id], axs[1])
# add text to image to show dice score
axs[1].text(0.5, 0.5, 'SAM DSC: {:.4f}'.format(ori_sam_dsc), fontsize=30, horizontalalignment='left', verticalalignment='top', color='yellow')
# axs[1].set_title('Mask with Untuned Model', fontsize=20)
axs[1].axis('off')

axs[2].imshow(imgs[img_id])
show_mask(medsam_segs[img_id], axs[2])
show_box(gtBbxes[img_id], axs[2])
# add text to image to show dice score
axs[2].text(0.5, 0.5, 'MedSAM DSC: {:.4f}'.format(medsam_dsc), fontsize=30, horizontalalignment='left', verticalalignment='top', color='yellow')
# axs[2].set_title('Ground Truth', fontsize=20)
axs[2].axis('off')
plt.show()  
plt.subplots_adjust(wspace=0.01, hspace=0)
# save plot
# plt.savefig(join(model_save_path, test_npzs[npz_idx].split('.npz')[0] + str(img_id).zfill(3) + '.png'), bbox_inches='tight', dpi=300)
plt.close()

In [ ]:
fPth=f'/kaggle/working/medSamSpine/sam_model_best.pth'

In [ ]:
%mv fPth /kaggle/input/npzdata564/

In [ ]:
import os
os.chdir('/kaggle/working/medSamSpine')
print(os.getcwd())
# print(os.("/kaggle/working"))
from IPython.display import FileLink
FileLink('/kaggle/working/medSamSpine/sam_model_best.pth')

In [ ]:
# set up the parser 设参
parser = argparse.ArgumentParser(description='preprocess CT images') #
parser.add_argument('-i', '--nii_path', type=str, default='/kaggle/input/vescos-dataset/verse518/verse518_CT.nii', help='path to the nii images') # 原图处
parser.add_argument('-gt', '--gt_path', type=str, default='/kaggle/input/vescos-dataset/verse518/verse518_CT-iso_seg.nii', help='path to the ground truth',) # 罩处
parser.add_argument('-o', '--npz_path', type=str, default='data/Npz_files', help='path to save the npz files') # 保存处(npz)

parser.add_argument('--image_size', type=int, default=256, help='image size') # 图大
parser.add_argument('--modality', type=str, default='CT', help='modality') # 模态
parser.add_argument('--anatomy', type=str, default='verSe', help='anatomy') # 解剖
parser.add_argument('--img_name_suffix', type=str, default='.nii.gz', help='image name suffix') # 图名后缀
parser.add_argument('--label_id', type=int, default=9, help='label id') # 标签id
parser.add_argument('--prefix', type=str, default='CT_Spine-verSe_', help='prefix') # 前缀
parser.add_argument('--model_type', type=str, default='vit_b', help='model type') # 模型类型
parser.add_argument('--checkpoint', type=str, default='/kaggle/input/segment-anything/pytorch/vit-b/1/model.pth', help='checkpoint') # 检查点
parser.add_argument('--device', type=str, default='cuda:0', help='device') # 设备
# seed
parser.add_argument('--seed', type=int, default=2023, help='random seed') # 随机种子
args = parser.parse_args()

prefix = args.modality + '_' + args.anatomy # 前缀
names = sorted(os.listdir(args.gt_path)) # 排序
names = [name for name in names if not os.path.exists(join(args.npz_path, prefix + '_' + name.split('.nii.gz')[0]+'.npz'))] # 保存处
names = [name for name in names if os.path.exists(join(args.nii_path, name.split('.nii.gz')[0] + args.img_name_suffix))] # 图名后缀


# split names into training and testing
np.random.seed(args.seed)
np.random.shuffle(names)
train_names = sorted(names[:int(len(names)*0.8)])
test_names = sorted(names[int(len(names)*0.8):])

# def preprocessing function
def preprocess_ct(gt_path, nii_path, gt_name, image_name, label_id, image_size, sam_model, device):
    gt_sitk = sitk.ReadImage(join(gt_path, gt_name))
    gt_data = sitk.GetArrayFromImage(gt_sitk)
    gt_data = np.uint8(gt_data==label_id)

    if np.sum(gt_data)>1000:
        imgs = []
        gts =  []
        imgEmds = []
        assert np.max(gt_data)==1 and np.unique(gt_data).shape[0]==2, 'ground truth should be binary'
        img_sitk = sitk.ReadImage(join(nii_path, image_name))
        image_data = sitk.GetArrayFromImage(img_sitk)
        # nii preprocess start
        lower_bound = -500
        upper_bound = 1000
        image_data_pre = np.clip(image_data, lower_bound, upper_bound) # 目的是将图像的像素值限制在一定范围内
        image_data_pre = (image_data_pre - np.min(image_data_pre))/(np.max(image_data_pre)-np.min(image_data_pre))*255.0 # 归一化
        image_data_pre[image_data==0] = 0 # 将图像中的0值像素置为0
        image_data_pre = np.uint8(image_data_pre) # 转为uint8

        z_index, _, _ = np.where(gt_data>0) # z轴上的索引
        z_min, z_max = np.min(z_index), np.max(z_index)

        for i in range(z_min, z_max):
            gt_slice_i = gt_data[i,:,:]
            gt_slice_i = transform.resize(gt_slice_i, (image_size, image_size), order=0, preserve_range=True, mode='constant', anti_aliasing=True) # 重采样: 保持原图像的像素值, 重采样到256x256, 最近邻插值, 常数填充, 抗锯齿
            if np.sum(gt_slice_i)>100:
                # resize img_slice_i to 256x256
                img_slice_i = transform.resize(image_data_pre[i,:,:], (image_size, image_size), order=3, preserve_range=True, mode='constant', anti_aliasing=True)
                # convert to three channels
                img_slice_i = np.uint8(np.repeat(img_slice_i[:,:,None], 3, axis=-1))
                assert len(img_slice_i.shape)==3 and img_slice_i.shape[2]==3, 'image should be 3 channels'
                assert img_slice_i.shape[0]==gt_slice_i.shape[0] and img_slice_i.shape[1]==gt_slice_i.shape[1], 'image and ground truth should have the same size'
                imgs.append(img_slice_i)
                assert np.sum(gt_slice_i)>100, 'ground truth should have more than 100 pixels' # 保证ground truth中有超过100个像素
                gts.append(gt_slice_i)
                if sam_model is not None:
                    sam_transform = ResizeLongestSide(sam_model.image_encoder.img_size)
                    resize_img = sam_transform.apply_image(img_slice_i)
                    # resized_shapes.append(resize_img.shape[:2])
                    resize_img_tensor = torch.as_tensor(resize_img.transpose(2, 0, 1)).to(device)
                    # model input: (1, 3, 1024, 1024)
                    input_image = sam_model.preprocess(resize_img_tensor[None,:,:,:]) # (1, 3, 1024, 1024)
                    assert input_image.shape == (1, 3, sam_model.image_encoder.img_size, sam_model.image_encoder.img_size), 'input image should be resized to 1024*1024'
                    # input_imgs.append(input_image.cpu().numpy()[0])
                    with torch.no_grad():
                        embedding = sam_model.image_encoder(input_image)
                        imgEmds.append(embedding.cpu().numpy()[0])

    if sam_model is not None:
        return imgs, gts, imgEmds
    else:
        return imgs, gts


#%% prepare the save path
save_path_tr = join(args.npz_path, prefix, 'train')
save_path_ts = join(args.npz_path, prefix, 'test')
os.makedirs(save_path_tr, exist_ok=True)
os.makedirs(save_path_ts, exist_ok=True)

#%% set up the model
sam_model = sam_model_registry[args.model_type](checkpoint=args.checkpoint).to(args.device)

for name in tqdm(train_names):
    image_name = name.split('.nii.gz')[0] + args.img_name_suffix
    gt_name = name
    imgs, gts, imgEmds = preprocess_ct(args.gt_path, args.nii_path, gt_name, image_name, args.label_id, args.image_size, sam_model, args.device)
    #%% save to npz file
    # stack the list to array
    if len(imgs)>1:
        imgs = np.stack(imgs, axis=0) # (n, 256, 256, 3)
        gts = np.stack(gts, axis=0) # (n, 256, 256)
        imgEmds = np.stack(imgEmds, axis=0) # (n, 1, 256, 64, 64)
        np.savez_compressed(join(save_path_tr, prefix + '_' + gt_name.split('.nii.gz')[0]+'.npz'), imgs=imgs, gts=gts, imgEmds=imgEmds)
        # save an example image for sanity check
        idx = np.random.randint(0, imgs.shape[0])
        img_idx = imgs[idx,:,:,:]
        gt_idx = gts[idx,:,:]
        bd = segmentation.find_boundaries(gt_idx, mode='inner')
        img_idx[bd, :] = [255, 0, 0]
        io.imsave(save_path_tr + '.png', img_idx, check_contrast=False)

# save testing data
for name in tqdm(test_names):
    image_name = name.split('.nii.gz')[0] + args.img_name_suffix
    gt_name = name
    imgs, gts = preprocess_ct(args.gt_path, args.nii_path, gt_name, image_name, args.label_id, args.image_size, sam_model=None, device=args.device)
    #%% save to npz file

In [ ]:
if len(imgs)>1:
        imgs = np.stack(imgs, axis=0) # (n, 256, 256, 3)
        gts = np.stack(gts, axis=0) # (n, 256, 256)
        imgEmds = np.stack(imgEmds, axis=0) # (n, 1, 256, 64, 64)
        np.savez_compressed(join(save_path_tr, prefix + '_' + gt_name.split('.nii.gz')[0]+'.npz'), imgs=imgs, gts=gts, imgEmds=imgEmds)

In [ ]:
ls

In [ ]:
from utils.SurfaceDice import compute_dice_coefficient
# set seeds
torch.manual_seed(2023)
np.random.seed(2023)

In [ ]:
#设置优化器，精调超参数, 以提高性能
lr = 1e-4 # 学习率
wd = 0 # 权重衰减
optimizer = torch.optim.Adam(sam_model.mask_decoder.parameters(), 
                             lr=lr, 
                             weight_decay=wd) # 优化器, 仅优化mask_decoder, 以提高性能

loss_fn = torch.nn.MSELoss() # 损失函数
# loss_fn = torch.nn.BCELoss()
keys = list(bbox_coords.keys()) # 所有的键

In [ ]:
from ipywidgets import interact
def skShow(img,
           xyz = 'Z',
           msk = None,
           bbx = None,
           lbs = None,
           title=None,
           margin=0.05,
           dpi=96,
           fSize=None,
           cmap="gray"):
    if isinstance(img, np.ndarray):
        nda = np.copy(img)
        img = sitk.GetImageFromArray(img)
    else:
        img = copy.deepcopy(img)
        if isinstance(img[0],bool):
            nda = np.zeros(img.shape, dtype=np.uint8)
            nda[img] = 1
        nda = sitk.GetArrayFromImage(img)
    if msk is not None:
        # ndaM = np.transpose(msk, (2, 1, 0))
        # msk[msk==0]=np.nan
        ndaM = sitk.GetArrayFromImage(msk)
        ndaM[ndaM==0] = np.nan

    spacing = img.GetSpacing()
    size = img.GetSize()
    if nda.ndim == 3: # 若3维数组
        # fastest dim, either component or x
        c = nda.shape[-1] # 通道数
        if c in (3, 4): # 若通道数为3或4, 则认为是2D图像
            nda = nda[:,:,0]
    elif nda.ndim == 4: # 若4维数组
        c = nda.shape[-1]
        if not c in (3, 4): # 若通道数不为3或4, 则认为是3Dv(4D)图像, 退出
            raise RuntimeError("Unable to show 3D-vector Image")
        else:
            # 去掉最后一维
            nda = nda[:,:,:,0]
    if nda.ndim == 2: # 若2维数组
        nda = nda[np.newaxis, ...] # nda增加后的维度为3维, 且最后一维为1
        ndaM = ndaM[np.newaxis, ..., np.newaxis] if msk is not None else None
        size = size + (1,) # size增加后的维度为3维, 且最后一维为1
        spacing = 1.
    # nda.shape = shape# nda的方向为LPS
    # size = nda.shape # size为z,y,x
    print('size:',size)
    xyzSize = [int(i+1)
                for i
                in (np.array(spacing)*np.array(size))
                ]
    sInd = {'X':2, 'Y':1, 'Z':0}[xyz]
    # sDic = dfDic(fdic)
    sDic = [dict(drt=['P==>A', 'L==>R'],
                        arr = nda, # nda的方向为LP
                        arrM = ndaM if msk is not None else None,
                        x = xyzSize[0],
                        y = xyzSize[1],
                        z = size[2],
                        extent = (0, xyzSize[0], 0, xyzSize[1]) # (left, right, bottom, top)
                        ),
            dict(drt=['I==>S', 'L==>R'],
                        arr=np.transpose(nda, (1,0,2)), # nda的方向为LS
                        arrM = np.transpose(ndaM, (1,0,2)) if msk is not None else None,
                        x = xyzSize[0],
                        y = xyzSize[2],
                        z = size[1],
                        extent = (0,xyzSize[0],xyzSize[2],0) # (left, right, bottom, top)
                        ),
            dict(drt=['I==>S', 'A<==P'],
                        arr=np.transpose(nda, (2,0,1)), # nda的方向为SP
                        arrM = np.transpose(ndaM, (2,0,1)) if msk is not None else None,
                        x = xyzSize[1],
                        y = xyzSize[2],
                        z = size[0],
                        extent = (0,xyzSize[1],xyzSize[2],0) # (left, right, bottom, top)
                        )][sInd]
    def callback(axe=None):
        figsize = (1 + margin) * sDic['y'] / dpi, (1 + margin) * sDic['x'] / dpi
        fig = plt.figure(figsize=[figsize, fSize][fSize is not None], dpi=dpi)
        ax = fig.add_axes([margin, margin, 1 - 2 * margin, 1 - 2 * margin])
        ax.imshow(sDic['arr'][axe, ...], extent=sDic['extent'], interpolation=None,  cmap=cmap) #, norm=SBWIN)
        if msk is not None:
            mArr = sDic['arrM'][axe, ...]
            
            ax.imshow(mArr, extent=sDic['extent'], interpolation=None, cmap=CMITK, alpha=0.3, vmin=1, vmax=64)
            if bbx is not None and xyz=='Z': # [TODO]总是定不准位置
                ls = np.unique(mArr)
                print('ls:', ls)
                for l in ls:
                    if not np.isnan(l):
                        def __bBx(mArr, label):
                            y, x = np.where(mArr == label) # x,y为msk中值为label的点的坐标
                            xy0 = np.array([np.min(x), np.min(y)])
                            xy1 = np.array([np.max(x), np.max(y)])
                            pad = np.random.randint(0, 10)
                            return xy0, xy1, pad
                        xy0, xy1, pad = __bBx(mArr, l)
                        x0, y0 = xy0
                        x1, y1 = xy1
                        lw = 2 + pad
                        z = axe
                        ax.add_patch(plt.Rectangle((x0, y0), x1-x0, y1-y0, edgecolor=COLORS[int(l-1)], facecolor=(0,0,0,0), lw=lw))
        if lbs is not None:
            llbs = np.unique(lbs)
            leg = ax.legend(handles=[plt.Rectangle((0, 0), 5, 5, color=COLORS[int(lb)]) for lb in llbs], labels=[LBLVS[int(lb)] for lb in llbs], loc='center left')

            leg_height = leg.get_frame().get_height()
            fig_height = fig.get_figheight()
            if leg_height > fig_height:
                fig.set_figheight(leg_height)  # Set fig height to match legend height
    # 在图像上标注坐标轴
        ax.set_ylabel(sDic['drt'][0])
        ax.set_xlabel(sDic['drt'][1])
        # 根据颜色标lv图例

        if title:
            plt.title(title)
        return plt.show()
    interact(callback, axe=(0, sDic['z'] - 1))

In [ ]:
def readImgSk(filePath=None, img = None, msk = False, img3C = True, RAS=False):
    '''sitk读图
    '''
    if img is None:
        # 若为文件夹
        # if os.path.isdir(filePath):
        if filePath.endswith('/'):
            img = readDcm(filePath)
        else:
            img = sitk.ReadImage(filePath)
    if RAS:
        imgRAS = (img)
    if msk is True:
        img = sitk.GetArrayFromImage(img)
        return img.astype(np.uint8)
        # if isinstance(img,sitk.Image):
        #     img = sitk.GetArrayFromImage(img)
        #     print('img.max():',img.max())
        # if img.max()>99.:
        #     print('Warning: mask image has value > 99')
        #     img = img//10/1.0
    if isinstance(img,np.ndarray):
        img=sitk.GetImageFromArray(img)
    img_vbCT_255 = sitk.Cast(sitk.RescaleIntensity(img), sitk.sitkUInt8)
    # 所有像素值都被10整除

    if img3C: # 三通道无法做魔法糖
        return sitk.GetArrayFromImage(sitk.Compose([img_vbCT_255]*3)) # 通道在最后
    else:
        return img_vbCT_255

In [ ]:
class GetMskBbx:
    ''' 罩机
        para:
            msk: 罩
            pad: 罩最厚
        retun:
            Dic: 罩字典
                'bBxes': Gt_xyxy, pad, z,
                'pBbx': pad_xyxy,
                'lbLy': label, layer,
                'lvBbx': lv_xyxy,
                'bBx': bBx_xyxy(optional)
    '''
    def __init__(self, msk, pad=20):
        """Initializes the GetBbx class."""
        if isinstance(msk, sitk.Image):
            self.mArr = sitk.GetArrayFromImage(msk)
        else:
            self.mArr = msk
        self.pad = pad
        self.Dic = dfDic(fdic)
        labels = np.unique(self.mArr)
        if len(labels) > 2:
            self.labels = labels[1:]
            self.get_3d_bbox()
        elif len(labels) == 2:
            label = labels[1]
            level = LBLVS[label]
            if len(self.mArr)==2:
                bBx, pBbx = self.get_2d_bbox(self.mArr[z],label)
                self.Dic['bBx'][level] = bBx
                self.Dic['pBbx'][level] = pBbx
            else:
                self.get_3d_bbox()
        else:
            raise ValueError('No labels found in mask.')
    def get_2d_bbox(self, arr2D, label, one=False):
        """Gets the 2D bounding box for a label."""
        y, x = np.where(arr2D == label)
        bBx = np.array([np.min(x), np.min(y), np.max(x), np.max(y)])
        h, w = arr2D.shape
#         print(h)
        px0=max(0, bBx[0] - np.random.randint(0, self.pad))
#         print(px0)
        py0=max(0, bBx[1] - np.random.randint(0, self.pad))
        px1=min(w, bBx[2] + np.random.randint(0, self.pad))
        py1=min(h, bBx[3] + np.random.randint(0, self.pad))
        return bBx, np.asarray([px0,py0,px1,py1])
    def get_3d_bbox(self):
        """Gets the 3D bounding box for a label."""
        for label in self.labels:
            z, y, x = np.where(self.mArr == label)
            z0, z1 = np.min(z), np.max(z)
            xyz0 = np.array([np.min(x), np.min(y), z0])
            xyz1 = np.array([np.max(x), np.max(y), z1])
            level = LBLVS[label]
            self.Dic['lvZs'][level]=np.array([z0, z1])
            print(level)
            for z in range(z0, z1 + 1):
                try:
                    bBx, pBbx = self.get_2d_bbox(self.mArr[z],label)
                    self.Dic['bBx'][level][z] = bBx
                    self.Dic['pBbx'][level][z] = pBbx
                except:
                    print(f'No 2D bbox found for slice {z-z0}')
                    self.Dic['bBx'][level][z] = None
                    self.Dic['pBbx'][level][z] = None

In [ ]:
bxDic = GetMskBbx(gts).Dic

In [ ]:
bbox_coords = bBxDic['bBx']['L1']
ground_truth_masks = {}
images = {}
for k in bbox_coords.keys():
    ground_truth_masks[k] = msks[k]
    images[k] = imgs[k]

In [ ]:
imgs=readImgSk(imgf)

In [ ]:
# 格式化图
from collections import defaultdict

import torch

from segment_anything.utils.transforms import ResizeLongestSide

transformed_data = defaultdict(dict)
for k in bbox_coords.keys():
    image = imgs[k]
    transform = ResizeLongestSide(sam_model.image_encoder.img_size) # 转换图像大小
    input_image = transform.apply_image(image)
    input_image_torch = torch.as_tensor(input_image, device=device) # 转换为张量
    transformed_image = input_image_torch.permute(2, 0, 1).contiguous()[None, :, :, :] # 转换为模型的输入格式

    input_image = sam_model.preprocess(transformed_image) # 预处理图像
    original_image_size = image.shape[:2] # 原始图像大小
    input_size = tuple(transformed_image.shape[-2:]) # 输入图像大小

    transformed_data[k]['image'] = input_image # 图像归典
    transformed_data[k]['input_size'] = input_size # 大小归典
    transformed_data[k]['original_image_size'] = original_image_size # 原始图像大小归典

In [9]:
### 医学影像预处理

#%% import
import SimpleITK as sitk
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from ipywidgets import interact
import pickle
import itertools as itt
from collections import defaultdict as dfDic
from ipywidgets import interact
import copy
from matplotlib.patches import Circle

import os
from matplotlib.colors import ListedColormap, Normalize
from matplotlib.patches import Circle
from scipy.ndimage import center_of_mass
import json



LBLVS = {
    1: 'C1', 2: 'C2', 3: 'C3', 4: 'C4', 5: 'C5', 6: 'C6', 7: 'C7',
    8: 'T1', 9: 'T2', 10: 'T3', 11: 'T4', 12: 'T5', 13: 'T6', 14: 'T7',
    15: 'T8', 16: 'T9', 17: 'T10', 18: 'T11', 19: 'T12', 20: 'L1',
    21: 'L2', 22: 'L3', 23: 'L4', 24: 'L5', 25: 'L6', 26: 'Sacrum',
    27: 'Cocc', 28: 'T13'
    
}

COLORS = (1/255)*np.array([
    [255,  0,  0], [  0,255,  0], [  0,  0,255], [255,255,  0], [  0,255,255],
    [255,  0,255], [255,239,213],  # Label 1-7 (C1-7)
    [  0,  0,205], [205,133, 63], [210,180,140], [102,205,170], [  0,  0,128],
    [  0,139,139], [ 46,139, 87], [255,228,225], [106, 90,205], [221,160,221],
    [233,150,122], [165, 42, 42],  # Label 8-19 (T1-12)
    [255,250,250], [147,112,219], [218,112,214], [ 75,  0,130], [255,182,193],
    [ 60,179,113], [255,235,205],  # Label 20-26 (L1-6, sacrum)
    [255,235,205], [255,228,196],  # Label 27 cocc, 28 T13,
    [218,165, 32], [  0,128,128], [188,143,143], [255,105,180],
    [255,  0,  0], [  0,255,  0], [  0,  0,255], [255,255,  0], [  0,255,255],
    [255,  0,255], [255,239,213],  # 29-39 unused
    [  0,  0,205], [205,133, 63], [210,180,140], [102,205,170], [  0,  0,128],
    [  0,139,139], [ 46,139, 87], [255,228,225], [106, 90,205], [221,160,221],
    [233,150,122],   # Label 40-50 (subregions)
    [255,250,250], [147,112,219], [218,112,214], [ 75,  0,130], [255,182,193],
    [ 60,179,113], [255,235,205], [255,105,180], [165, 42, 42], [188,143,143],
[255,235,205], [255,228,196], [218,165, 32], [  0,128,128], [30,14,255]# rest unused
    ])
CMITK = ListedColormap(COLORS)
CMITK.set_bad(color='w', alpha=0)  # set NaN to full opacity for overlay
SBWIN = Normalize(vmin=-500, vmax=1300, clip=True) # 软骨窗是-500到1300, 用于显示软骨
HBWIN = Normalize(vmin=-200, vmax=1000, clip=True) # 骨窗是-200到1000, 用于显示骨头


def fdic():
    return dfDic(fdic)


def pk2file(file, data=None):
    # if not file.endswith('.pickle' or '.pkl'):
    #     file = file+'.pickle'
    if data is not None:
        with open(file, 'wb') as f:
            pickle.dump(data, f)
    else:
        with open(file, 'rb') as f:
            data = pickle.load(f)
        return data

def readDcm(dcm_path):
    """
    读取DICOM图像序列,返回3D Image对象
    """
    # 读取DICOM图像序列
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(dcm_path)
    reader.SetFileNames(dicom_names)

    # 注意,此时还没有加载图像数据
    # 拼接3D图像
    image3d = reader.Execute()

    # 注意,如果图像维度<=2,则执行后image3d仍为None
    # 所以需要检验
    if image3d is None:
        print('Error: less than 3 dimensions')
        return None

    # 获得图像的元数据
    img_array = sitk.GetArrayFromImage(image3d)
    print('Image data type: ', image3d.GetPixelIDTypeAsString())
    print('Image size: ', image3d.GetSize())
    print('Image spacing: ', image3d.GetSpacing())
    print('Image origin: ', image3d.GetOrigin())
    print('Image dimension: ', image3d.GetDimension())

    return image3d
#%%
def readImgSk(filePath=None, img = None, msk = False, img3C = True):
    '''sitk读图
    '''
    if img is None:
        # 若为文件夹
        # if os.path.isdir(filePath):
        if filePath.endswith('/'):
            img = readDcm(filePath)
        else:
            img = sitk.ReadImage(filePath)
    if msk is True:
        return img
        # if isinstance(img,sitk.Image):
        #     img = sitk.GetArrayFromImage(img)
        #     print('img.max():',img.max())
        # if img.max()>99.:
        #     print('Warning: mask image has value > 99')
        #     img = img//10/1.0
    if isinstance(img,np.ndarray):
        img=sitk.GetImageFromArray(img)
    img_vbCT_255 = sitk.Cast(sitk.RescaleIntensity(img), sitk.sitkUInt8)
    # 所有像素值都被10整除

    if img3C: # 三通道无法做魔法糖
        return sitk.GetArrayFromImage(sitk.Compose([img_vbCT_255]*3)) # 通道在最后
    else:
        return img_vbCT_255
#%%
def imgRAS(image):
    """Reorients an image to standard radiology view."""

    dir = np.array(image.GetDirection()).reshape(len(image.GetSize()), -1)
    ind = np.argmax(np.abs(dir), axis=0)
    new_size = np.array(image.GetSize())[ind]
    new_spacing = np.array(image.GetSpacing())[ind]
    new_extent = new_size * new_spacing
    new_dir = dir[:, ind]

    flip = np.diag(new_dir) < 0  #
    flip_diag = flip * -1
    flip_diag[flip_diag == 0] = 1
    flip_mat = np.diag(flip_diag)

    new_origin = np.array(image.GetOrigin()) + np.matmul(new_dir, (new_extent * flip))
    new_dir = np.matmul(new_dir, flip_mat)

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(new_spacing.tolist())
    resample.SetSize(new_size.tolist())
    resample.SetOutputDirection(new_dir.flatten().tolist())
    resample.SetOutputOrigin(new_origin.tolist())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(image.GetPixelIDValue())
    resample.SetInterpolator(sitk.sitkNearestNeighbor)

    return resample.Execute(image)

def cropOtsu(image):
    ''' Otsu阈值裁图
    '''
    inside_value = 0
    outside_value = 255
    label_shape_filter = sitk.LabelShapeStatisticsImageFilter() # 用于计算图像的轴对齐边界框。
    label_shape_filter.Execute(sitk.OtsuThreshold(image, inside_value, outside_value))
    bounding_box = label_shape_filter.GetBoundingBox(outside_value) # 获取边界框
    print('bounding_box', bounding_box)
    return sitk.RegionOfInterest(image,
                                 bounding_box[int(len(bounding_box) / 2) :],
                                 bounding_box[0 : int(len(bounding_box) / 2)],)

def sk3C2gd(img):
    # 三通道转灰度
    if img.GetDimension() == 3:
        return (sitk.VectorIndexSelectionCast(img, 0) +
                sitk.VectorIndexSelectionCast(img, 1) +
                sitk.VectorIndexSelectionCast(img, 2)) / 3


def orientDic(img):
    return {'X': 'L-R'[img.GetDirection()[0]+1],
            'Y': 'P-A'[img.GetDirection()[4]+1],
            'Z': 'I-S'[img.GetDirection()[8]+1]}

#%%

#%%

#%%

def bBx2xy(bBx: np.ndarray) -> np.ndarray:
    xywh = np.asarray(bBx)
    dHf = len(xywh)//2
    ini, fin = xywh[:dHf],xywh[:dHf]+xywh[dHf:]
    return np.asarray(ini.tolist()+fin.tolist())

#Q: 已知xyxy1和xyxy2, iou是怎么计算出来的?
#A: 1. 计算两个框的面积, 2. 计算两个框的交集, 3. 计算iou
def getIou(xyxy1: list, xyxy2: list):
    box1_area = (xyxy1[2] - xyxy1[0]) * (xyxy1[3] - xyxy1[1])
    box2_area = (xyxy2[2] - xyxy2[0]) * (xyxy2[3] - xyxy2[1])
    interSection = max(0, min(xyxy1[2], xyxy2[2]) - max(xyxy1[0], xyxy2[0])) * max(0, min(xyxy1[3], xyxy2[3]) - max(xyxy1[1], xyxy2[1]))
    iou = interSection / (box1_area + box2_area - interSection)
    return iou

def svAllSam(sData=None, sam=True, cId=0): # 将全分的结果转为sv.Dets
    '''
    Notes:

    '''

    if isinstance(sData[0], dict):
        sortedsData = sorted(
            sData, key=lambda x: x["area"], reverse=True
        )
        xywh = np.array([mask["bbox"] for mask in sortedsData])
        masks = np.array([mask["segmentation"] for mask in sortedsData])
        # 若masks为bool型, 则转为int型
        masks = getMsk01(masks)
        confidence = np.array([mask["predicted_iou"] for mask in sortedsData])
        tracker_id = np.arange(len(sortedsData))
        class_id = np.ones(len(sortedsData))*cId
        return sv.Detections(xyxy=bBx2xy(xywh),
                            mask=masks,
                            confidence=np.array(confidence),
                            tracker_id=tracker_id,
                            class_id=class_id
                            )
    elif isinstance(sData[0], list):
        for i, data in enumerate(sData):
            if sam:
                data = svAllSam(data, cId=i)
            if i == 0:
                dets = sv.Detections(xyxy=data.xyxy,
                                    mask=data.mask,
                                    confidence=data.confidence,
                                    tracker_id=data.tracker_id,
                                    class_id=data.class_id
                                    )

            else:
                dets.xyxy=np.append(
                    dets.xyxy,data.xyxy, axis=0)
                dets.mask=np.append(
                    dets.mask,data.mask, axis=0)
                dets.confidence=np.append(
                    dets.confidence,data.confidence, axis=0)
                dets.tracker_id=np.append(
                    dets.tracker_id,data.tracker_id, axis=0)
                dets.class_id=np.append(
                    dets.class_id,data.class_id, axis=0)
        return dets

def getMsk01(mArr):
    if isinstance(mArr, sitk.Image):
        mArr = sitk.GetArrayFromImage(mArr>0)
    if isinstance(mArr[0], bool):
        mArr = np.where(mArr == True, 1, 0)
        # 若masks的不是[0,1]型, 则转为[0,1]型
    elif mArr.max>1:
        mArr = np.where(mArr>0, 1, 0)
    return mArr

def bBin(arr):
    return np.where(arr == True, 1, 0)
def bBool(arr, thr=0):
    return np.where(arr>thr, True, False)
#%%
#%%
def dataIso(img_nib, msk_nib, ctdList, spc=(1,1,1), axes='IPL'):
    # Resample and Reorient data
    if spc is None:
        spc = img_nib.header.get_zooms()
        mSpc = msk_nib.header.get_zooms()
        img_iso = img_nib
        msk_iso = [msk_nib, resample_mask_to(msk_nib, img_iso)][mSpc==spc]
        ctds = [ctdList, rescale_centroids(ctdList, img_nib, spc)][mSpc==spc]
    else:
        img_iso = resample_nib(img_nib, voxel_spacing=spc, order=3)
        msk_iso = resample_nib(msk_nib, voxel_spacing=spc, order=0)
        ctds = rescale_centroids(ctdList, img_nib, spc) # 质心缩放
    img_iso = reorient_to(img_iso, axcodes_to=axes)
    msk_iso = reorient_to(msk_iso, axcodes_to=axes)
    ctds = reorient_centroids_to(ctds, img_iso)
    return img_iso, msk_iso, ctds




#%%
def end():

    pass

In [ ]:
LBLVS[12]

In [ ]:
LBLVS.get('T5')

In [ ]:
type(LBLVS)

In [ ]:

import os
import logging
import math
# import fvcore.nn.weight_init as weight_init
import torch
import torch.nn as nn

from torch import Tensor
from .image_encoder import ImageEncoderViT

logger = logging.getLogger(__name__)


__all__ = ["LoRA_SAM"]


class _LoRA_qkv_timm(nn.Module):
    """In timm it is implemented as
    self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
    B, N, C = x.shape
    qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim).permute(2, 0, 3, 1, 4)
    q, k, v = qkv.unbind(0)
    """

    def __init__(
        self,
        qkv: nn.Module,
        linear_a_q: nn.Module,
        linear_b_q: nn.Module,
        linear_a_v: nn.Module,
        linear_b_v: nn.Module,
    ):
        super().__init__()
        self.qkv = qkv
        self.linear_a_q = linear_a_q
        self.linear_b_q = linear_b_q
        self.linear_a_v = linear_a_v
        self.linear_b_v = linear_b_v
        self.dim = qkv.in_features
        self.w_identity = torch.eye(qkv.in_features)

    def forward(self, x):
        qkv = self.qkv(x)  # B,N,3*org_C
        new_q = self.linear_b_q(self.linear_a_q(x))
        new_v = self.linear_b_v(self.linear_a_v(x))
        qkv[:, :, :, : self.dim] += new_q
        qkv[:, :, :, -self.dim :] += new_v
        return qkv

class LoRA_Encoder(nn.Module):
    def __init__(self, vit_model: ImageEncoderViT, r: int, topk: int, lora_layer=None):
        super(LoRA_Encoder, self).__init__()

        topk = max(min(len(vit_model.blocks), topk), 1)

        assert r > 0
        if lora_layer:
            self.lora_layer = lora_layer
        else:
            self.lora_layer = list(range(len(vit_model.blocks)))[-topk:]

        # dim = vit_model.head.in_features
        # create for storage, then we can init them or load weights
        self.w_As = nn.ModuleList()  # These are linear layers
        self.w_Bs = nn.ModuleList()

        # Here, we do the surgery
        for t_layer_i, blk in enumerate(vit_model.blocks):
            # If we only want few lora layer instead of all
            if t_layer_i not in self.lora_layer:
                continue
            w_qkv_linear = blk.attn.qkv
            self.dim = w_qkv_linear.in_features
            w_a_linear_q = nn.Linear(self.dim, r, bias=False)
            w_b_linear_q = nn.Linear(r, self.dim, bias=False)
            w_a_linear_v = nn.Linear(self.dim, r, bias=False)
            w_b_linear_v = nn.Linear(r, self.dim, bias=False)
            self.w_As.append(w_a_linear_q)
            self.w_Bs.append(w_b_linear_q)
            self.w_As.append(w_a_linear_v)
            self.w_Bs.append(w_b_linear_v)
            blk.attn.qkv = _LoRA_qkv_timm(
                w_qkv_linear,
                w_a_linear_q,
                w_b_linear_q,
                w_a_linear_v,
                w_b_linear_v,
            )
        self.reset_parameters()
        self.lora_vit = vit_model

        # freeze image encoder vit
        for _, p in self.lora_vit.neck.named_parameters():
            p.requires_grad = False



    def reset_parameters(self) -> None:
        for w_A in self.w_As:
            nn.init.kaiming_uniform_(w_A.weight, a=math.sqrt(5))
        for w_B in self.w_Bs:
            nn.init.zeros_(w_B.weight)

    def forward(self, x: Tensor) -> Tensor:
        return self.lora_vit(x)
    




In [ ]:
%cd /kaggle/working/MedSAM/segment_anything/modeling